In [1]:
import glob, os, sys, shutil
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn as skl
import xgboost




%load_ext autoreload
%autoreload 2
%matplotlib inline
if os.name == 'posix':
    %config InlineBackend.figure_format = 'retina' #retina display settings

In [39]:
from sklearn import preprocessing
from tpot import TPOTClassifier

In [40]:
df = pd.read_csv(filepath_or_buffer='./train.csv')
df_test = pd.read_csv(filepath_or_buffer='./test.csv')

XX = pd.concat([df.drop('Survived', axis = 1), df_test], axis = 0)
XX.drop('PassengerId', axis = 1, inplace=True)

XX['Nulls'] = XX.Cabin.isnull().astype('int') + XX.Age.isnull().astype('int')
XX['FamilySize'] = XX['SibSp'] + XX['Parch'] + 1

import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

XX['Title'] = XX['Name'].apply(get_title)
XX['Title'] = XX['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                             'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
XX['Title'] = XX['Title'].replace('Mlle', 'Miss')
XX['Title'] = XX['Title'].replace('Ms', 'Miss')
XX['Title'] = XX['Title'].replace('Mme', 'Mrs')
XX.drop(['Name'], inplace = True, axis = 1)
label_title = preprocessing.LabelEncoder()
label_title.fit(XX.Title)
XX.Title = label_title.transform(XX.Title)


XX['Cabin_mapped'] = XX['Cabin'].astype(str).str[0] # this captures the letter
cabin_dict = {k:i for i, k in enumerate(XX.Cabin_mapped.unique())} 
XX.loc[:, 'Cabin_mapped'] = XX.loc[:, 'Cabin_mapped'].map(cabin_dict)
XX.drop(['Cabin'], inplace = True, axis = 1)


XX['Fare'].fillna(XX.Fare.median(), inplace = True)
scaler_box = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True)
scaler_box.fit(XX.Fare.values.reshape(-1, 1))
XX.Fare = scaler_box.transform(XX.Fare.values.reshape(-1,1))

XX['Age'].fillna(XX.Age.median(), inplace = True)
scaler_box = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True)
scaler_box.fit(XX.Age.values.reshape(-1, 1))
XX.Age = scaler_box.transform(XX.Age.values.reshape(-1,1))





XX['Embarked'].fillna('S', inplace = True)
XX['Embarked'] = XX['Embarked'].map( {'C': 1, 'S': 0, 'Q':2} ).astype(int)

XX['Sex'] = XX['Sex'].map( {'female': 1, 'male': 0} ).astype(int)


XX.drop(['Ticket'], inplace = True, axis = 1)
# XX.drop(['Parch'], inplace = True, axis = 1)
# XX.drop(['SibSp'], inplace = True, axis = 1)

# X_dummies = pd.get_dummies(XX, columns = ['Title'], drop_first= True)


# print(XX.columns)
XX.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Nulls,FamilySize,Title,Cabin_mapped
0,3,0,-0.539691,1,0,-0.905144,0,1,2,2,0
1,1,1,0.681592,1,0,1.319970,1,0,2,3,1
2,3,1,-0.220411,0,0,-0.812033,0,1,1,1,0
3,1,1,0.462678,1,0,1.060890,0,0,2,3,1
4,3,0,0.462678,0,0,-0.795673,0,1,1,2,0


In [41]:
X = XX[:len(df)];
new_X = XX[len(df):]
y = df.Survived

In [42]:
# X_train, X_test, y_train, y_test = skl.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=50)
tpot.fit(X, y)

Generation 1 - Current best internal CV score: 0.8462979650590631
Generation 2 - Current best internal CV score: 0.8462979650590631
Generation 3 - Current best internal CV score: 0.8496562683635325
Generation 4 - Current best internal CV score: 0.8496814475703307
Generation 5 - Current best internal CV score: 0.8496814475703307
Generation 6 - Current best internal CV score: 0.8496814475703307
Generation 7 - Current best internal CV score: 0.8519034593747683
Generation 8 - Current best internal CV score: 0.8519034593747683
Generation 9 - Current best internal CV score: 0.8519034593747683
Generation 10 - Current best internal CV score: 0.8519034593747683
Generation 11 - Current best internal CV score: 0.8519034593747683
Generation 12 - Current best internal CV score: 0.8519034593747683
Generation 13 - Current best internal CV score: 0.8519034593747683
Generation 14 - Current best internal CV score: 0.8519034593747683
Generation 15 - Current best internal CV score: 0.8552616208245517
Gene

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=1000000,
        max_eval_time_mins=5, max_time_mins=50, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=100,
        random_state=None, scoring=None, subsample=1.0, template=None,
        use_dask=False, verbosity=2, warm_start=False)

In [44]:
y_predict = tpot.predict(X)

In [45]:
(y_predict==y).mean()

0.9225589225589226

In [47]:
tpot.export('tpot_pipeline_2.py')


In [ ]:
y_tpot_prediction = tpot.predict(new_X)
submission = pd.concat([df_test.PassengerId, pd.DataFrame(y_tpot_prediction)], axis = 'columns')
submission.columns = ["PassengerId", "Survived"]
submission.to_csv('tpot_03.csv', header=True, index = False)

### OTHER 

In [52]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
clf = svm.SVC()

clf.fit(X, y)

C:\Users\aamico\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [53]:
y_svm_prediction = clf.predict(new_X)
submission = pd.concat([df_test.PassengerId, pd.DataFrame(y_svm_prediction)], axis = 'columns')
submission.columns = ["PassengerId", "Survived"]
submission.to_csv('svm_01.csv', header=True, index = False)

In [57]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X, y)

y_rfc_prediction = rfc.predict(new_X)
submission = pd.concat([df_test.PassengerId, pd.DataFrame(y_rfc_prediction)], axis = 'columns')
submission.columns = ["PassengerId", "Survived"]
submission.to_csv('rfc_01.csv', header=True, index = False)

C:\Users\aamico\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [62]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()
ridge.fit(X, y)

y_ridge_prediction = ridge.predict(new_X)
submission = pd.concat([df_test.PassengerId, pd.DataFrame(y_ridge_prediction)], axis = 'columns')
submission.columns = ["PassengerId", "Survived"]
submission.to_csv('ridge_01.csv', header=True, index = False)



In [63]:
y_ridge_prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,